# Overview

# Imports

In [1]:
import requests

In [2]:
requests.__version__

'2.31.0'

# Example 1 - querying PubMed and retrieving a list of IDs matching the search

**References and guides**

1. The base query, some example searches, and XML output parameters can be found here:
https://www.ncbi.nlm.nih.gov/books/NBK25500/

2. Parameters and syntax in depth:
https://www.ncbi.nlm.nih.gov/books/NBK25499/

3. These are the PubMed specific search term fields and their tags that should be used to construct the query:
https://pubmed.ncbi.nlm.nih.gov/help/#tiab

4. As seen in the url_suffix variable, it uses an 'Esearch' utility. There are a total of 9 functionalities under eutils (see base url). Others can be found here:
https://www.ncbi.nlm.nih.gov/books/NBK25497/

**Notes**
* This API returns an XML output and not a json. 
* By default, it will only return 20 IDs.
* The Count parameter within the XML output shows the total number of records matching the search query.

In [30]:
base_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'

In [35]:
database = 'pubmed'

In [41]:
query = 'glioblastoma[tiab]+AND+ALK[tiab]'

In [42]:
url_suffix = f'esearch.fcgi?db={database}&term={query}'

In [43]:
url_suffix

'esearch.fcgi?db=pubmed&term=glioblastoma[tiab]+AND+ALK[tiab]'

In [44]:
final_url = base_url + url_suffix

In [45]:
final_url

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=glioblastoma[tiab]+AND+ALK[tiab]'

In [46]:
response = requests.get(final_url)

* This code means the request was successful. There are different codes such as 400 which means bad request.

In [47]:
response.status_code

200

In [50]:
response.encoding

'UTF-8'

* As expected, it gives an XML output.
* Also, the number of records was confirmed by searching manually on PubMed: https://pubmed.ncbi.nlm.nih.gov/?term=%28ALK%5BTitle%2FAbstract%5D%29+AND+%28glioblastoma%5BTitle%2FAbstract%5D%29&sort=

In [55]:
print(response.text)

<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">
<eSearchResult><Count>76</Count><RetMax>20</RetMax><RetStart>0</RetStart><IdList>
<Id>38285799</Id>
<Id>37939020</Id>
<Id>37861443</Id>
<Id>37271069</Id>
<Id>37260294</Id>
<Id>37240478</Id>
<Id>37168365</Id>
<Id>36823756</Id>
<Id>36780194</Id>
<Id>36707425</Id>
<Id>35724395</Id>
<Id>35625997</Id>
<Id>35190826</Id>
<Id>34702773</Id>
<Id>34626238</Id>
<Id>34341009</Id>
<Id>34323181</Id>
<Id>34015889</Id>
<Id>33966367</Id>
<Id>33887544</Id>
</IdList><TranslationSet/><QueryTranslation>"glioblastoma"[Title/Abstract] AND "ALK"[Title/Abstract]</QueryTranslation></eSearchResult>

